* creates `deriv/timing_df_recall.csv`
- make and save a dataframe containing onset and offset TRs for each cue presentation during recall

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl

from nilearn import image, plotting, input_data
from scipy.spatial import distance

pd.options.display.max_rows = 200

# TODO
- include TR-level recall transcription information from `behav/from_silvy/recallTranscriptions/S%i.csv`
- include column with path taken on given wedding


In [2]:
""" 
load logfile into dataframe
"""

def include_offset_TR_col(df):
  df = df.sort_values('onset_TR')
  df.iloc[:-1].loc[:,'offset_TR'] = df.iloc[1:].onset_TR.values
  df.iloc[-1].offset_TR = 999
  return df

def init_logfile_df(sub_num):
  """ 
  read logfile into dataframe
  """
  TR_rate = 1.5
  # load sub log file
  fpath = 'data/behav/silvy_buckets/sub%.2iday2/%i_recall_mylog.log'%(100+sub_num,sub_num)
  f = open(fpath, "r")
  first_line = f.readline()
  # extract first TR tstamp
  first_tstamp = float(first_line.split()[1])
  # init df
  sub_df = pd.DataFrame(columns=['sub_num','wed_id','onset_TR','offset_TR'])
  # loop over rows in logfile (stim onsets)
  for x in f:
    # extract info
    RV,tstamp,stim = x.split(' ')
    tstamp = float(tstamp[:-1])
    wed_id = stim.split('/')[-1].split('.')[0][1:]
    onset_TR = np.floor((tstamp-first_tstamp)/TR_rate)
    # populate df with info
    sub_df.loc[tstamp,'sub_num'] = sub_num
    sub_df.loc[tstamp,'wed_id'] = wed_id
    sub_df.loc[tstamp,'onset_TR'] = onset_TR 
  return sub_df

def expand_logdf(logdf):
  """
  expand logdf so that each row is a TR
  """
  logdf_expand = pd.DataFrame(columns=['sub_num','wed_id','onsetTR','offsetTR'])
  for idx,logrow in logdf.iterrows():  
    sub_num,wed_id,onsetTR,offsetTR = logrow
    for TR in np.arange(onsetTR,offsetTR):
      logdf_expand.loc[TR,'sub_num']=sub_num
      logdf_expand.loc[TR,'wed_id']=wed_id
      logdf_expand.loc[TR,'onsetTR']=onsetTR
      logdf_expand.loc[TR,'offsetTR']=offsetTR
  return logdf_expand

def load_logdf(sub_num):
  """ wrapper 
  each row is a TR
  cols: sub_num,wed_id,wed_onsetTR,wed_offset
  """
  logdf = init_logfile_df(sub_num)
  logdf = include_offset_TR_col(logdf)
  logdf.index = logdf.onset_TR.astype(int)
  logdf = logdf.astype(int)
  logdf = expand_logdf(logdf)
  return logdf

### include transcript information

In [3]:
def load_transcript_df(sub_num):
  """ 
  reindex from seconds to TRs
    NB this causes duplicates
    to resolve duplicates, non-zero entries break the tie
  """
  tdf = pd.read_csv('data/behav/silvy_buckets/recallTranscriptions/S%i.csv'%sub_num,index_col=0).T.fillna(0)
  if sub_num==6: tdf = tdf.replace('\n',0)   
  # transform index from seconds to TRs
  TR_rate = 1.5
  tdf.index = (tdf.index.astype(int)/TR_rate).astype(int)
  tdf = tdf.astype(int)
  return resolve_tdf_duplicates(tdf)

def resolve_tdf_duplicates(tdf):
  """
  when converting tdf seconds to TRs,
    there are duplicates
    to resolve duplicates, 
      take row with most non-zeros
      *NB could be improved*
  """
  L = []
  for idx in tdf.index.unique():
    rows = tdf.loc[idx,:]
    # detect if there are duplicates
    if len(rows.shape)==1:
      row = rows
    elif len(rows.shape)>1:
      # resolve duplicate entries
      keep_row_num = np.sum(rows != 0,1).argmax()    
      row = rows.iloc[keep_row_num]
    L.append(row)
  return pd.concat(L,1).T

tdf = load_transcript_df(33)


### form recalldf
- initialize recalldf from logdf
- include recall column by combining information from transcriptdf
- include path column 
    - where is path information found?
    - NB path can be infered from state+schema

In [4]:
def make_sub_recall_df(ldf,tdf):
  """ for given subject
  combine ldf and tdf into recall_df
  contains onsetTR,offsetTR of each wed_id 
    and the recall code for each TR
  """
  ldf.loc[:,'recall'] = 99
  for TR,ldf_row in ldf.iterrows():
    # find relevant entry in transcript df
    tdf_row_num = TR-int(ldf_row.onsetTR)
    wed_id = ldf_row.wed_id
    # this breaks the loop if TR exceeds valid row value in tdf
    # tdf only has 160 rows
    if tdf_row_num > 160:
        continue
    # extract recall from transcript_df
    recall_value = tdf.loc[tdf_row_num,'W%i'%wed_id]
    ldf.loc[TR,'recall']=recall_value
  # remove entries with no recall information
  recall_df = ldf[ldf.recall!=99]
  return recall_df


In [5]:
ALL_SUB_NS = [
  3,4,5,6,7,8,9,10,
  12,13,14,15,17,18,19,
  22,23,24,25,26,27,28,29,
  30,31,32,33,34,35,36,38,
  40,41,42,43,44
  ]

In [6]:
""" loop over subejcts to make group recall_df"""
L = []
for sub_num in ALL_SUB_NS:
  print('sub',sub_num)
  ldf = load_logdf(sub_num)
  tdf = load_transcript_df(sub_num)
  sub_recall_df = make_sub_recall_df(ldf,tdf)
  L.append(sub_recall_df)

sub 3


//anaconda3/envs/brain/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
//anaconda3/envs/brain/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


sub 4


//anaconda3/envs/brain/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
//anaconda3/envs/brain/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


sub 5


...

//anaconda3/envs/brain/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
//anaconda3/envs/brain/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


sub 43


//anaconda3/envs/brain/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
//anaconda3/envs/brain/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


sub 44


//anaconda3/envs/brain/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
//anaconda3/envs/brain/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
recall_df = pd.concat(L)
recall_df

,sub_num,wed_id,onsetTR,offsetTR,recall
0,3,22,0,57,0
1,3,22,0,57,0
2,3,22,0,57,0
3,3,22,0,57,0
4,3,22,0,57,0
...,...,...,...,...,...
468,44,17,312,999,0
469,44,17,312,999,0
470,44,17,312,999,0
471,44,17,312,999,0


In [8]:
recall_df[recall_df.sub_num == 34].iloc[250:450]

,sub_num,wed_id,onsetTR,offsetTR,recall
250,34,38,234,278,0
251,34,38,234,278,0
252,34,38,234,278,0
253,34,38,234,278,0
254,34,38,234,278,0
255,34,38,234,278,0
256,34,38,234,278,0
257,34,38,234,278,0
258,34,38,234,278,6
259,34,38,234,278,6


In [9]:
recall_df.to_csv('deriv/recall_df.csv')

In [11]:
""" 
NB THERE ARE STILL SOME 999 ENTRIES AT THE END WITH NO INFO
  because I do not have information about the length of the imaging run
  for the final wedding
"""
recall_df[recall_df.sub_num == 34].iloc[-100:]

,sub_num,wed_id,onsetTR,offsetTR,recall
544,34,17,483,999,0
545,34,17,483,999,0
546,34,17,483,999,0
547,34,17,483,999,0
548,34,17,483,999,0
549,34,17,483,999,0
550,34,17,483,999,0
551,34,17,483,999,0
552,34,17,483,999,0
553,34,17,483,999,0
